# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс.

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [1]:
import csv


def lazy_read(chunksize):
    with open('data/recipes_full.csv', encoding = 'utf-8') as f:
        reader = csv.reader(f, delimiter = ',')
        next(reader)
        rows = []
        for i, row in enumerate(reader):
            if i and i % chunksize == 0:
                yield rows
                rows = []
            for tag in row[5].replace('[', '').replace(']', '').replace("'", '').split(','):
                tag = tag.strip()
                if tag:
                    rows.append((row[1], tag, row[6]))
    yield rows

In [2]:
header = ['id', 'tag', 'n_steps']
for i, rows in enumerate(lazy_read(2350000 // 8)):
    print(i)
    with open(f'tmp/id_tag_nsteps_{i}.csv', 'w', newline = '') as f:
        writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONE)
        writer.writerow(header)
        writer.writerows(rows)

0
1
2
3
4
5
6
7


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [3]:
import pandas as pd


def get_dict_avg_n_steps(filename):
    data = pd.read_csv(filename, sep=';')
    return data.groupby('tag')['n_steps'].mean().to_dict()


get_dict_avg_n_steps('tmp/id_tag_nsteps_0.csv')

{'1-day-or-more': 4.599847850893876,
 '15-minutes-or-less': 4.982572410407462,
 '3-steps-or-less': 4.686890064667843,
 '30-minutes-or-less': 7.573613963039015,
 '4-hours-or-less': 10.123393316195372,
 '5-ingredients-or-less': 5.37246680642907,
 '60-minutes-or-less': 9.323185210312076,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.5041736227045077,
 'a1-sauce': 3.4912207357859533,
 'african': 4.286177105831533,
 'american': 7.5644070344205625,
 'amish-mennonite': 3.59009009009009,
 'angolan': 3.5065897858319603,
 'appetizers': 6.244792699861138,
 'apples': 4.770433083686096,
 'april-fools-day': 3.551245406288281,
 'argentine': 3.5860145860145862,
 'artichoke': 3.544672131147541,
 'asian': 6.433729101041919,
 'asparagus': 4.0344827586206895,
 'australian': 4.247632920611799,
 'austrian': 3.57312925170068,
 'avocado': 3.459277917716205,
 'bacon': 4.076719576719577,
 'baja': 3.498955286251567,
 'baked-beans': 3.498090793381417,
 'baking': 3.576327896277708,
 '

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.

In [4]:
def get_dict_avg_n_steps_all(filenames):
    count = 0
    summa = 0
    for filename in filenames:
        data = pd.read_csv(filename, sep=';')
        group = data.groupby('tag')
        count += group['n_steps'].count()
        summa += group['n_steps'].sum()
    mean = summa / count
    return mean.to_dict()

In [5]:
filenames = []
for i in range(8):
    filenames.append(f'tmp/id_tag_nsteps_{i}.csv')

In [6]:
%timeit get_dict_avg_n_steps_all(filenames)

5.42 s ± 58.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
get_dict_avg_n_steps_all(filenames)

{'1-day-or-more': 4.459439233685891,
 '15-minutes-or-less': 4.9865173969072165,
 '3-steps-or-less': 4.728792059112676,
 '30-minutes-or-less': 7.610044050452816,
 '4-hours-or-less': 10.065202927478376,
 '5-ingredients-or-less': 5.345920719916704,
 '60-minutes-or-less': 9.413654300607185,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.5116482077581974,
 'a1-sauce': 3.5280419809773695,
 'african': 4.371965990595164,
 'american': 7.575099698868723,
 'amish-mennonite': 3.560696788241698,
 'angolan': 3.4931362780914426,
 'appetizers': 6.231026464978969,
 'apples': 4.85682676038006,
 'april-fools-day': 3.5103864194773284,
 'argentine': 3.5665815193247448,
 'artichoke': 3.4960065645514224,
 'asian': 6.445874984183222,
 'asparagus': 4.047421217955213,
 'australian': 4.218603314493725,
 'austrian': 3.582217343578485,
 'avocado': 3.531110989071338,
 'bacon': 4.10665269251884,
 'baja': 3.53721200066302,
 'baked-beans': 3.4936835106382977,
 'baking': 3.6306821245618766,

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [8]:
%%file 'zad_4.py'
import pandas as pd


def get_count_sum(file):
    data = pd.read_csv(file, sep=';')
    group = data.groupby('tag')
    count = group['n_steps'].count()
    summa = group['n_steps'].sum()
    return count, summa


def sum_results(results):
    count = 0
    summa = 0
    for data in results:
        count += data[0]
        summa += data[1]
    mean = summa / count
    return mean.to_dict()

Overwriting zad_4.py


In [9]:
from zad_4 import get_count_sum, sum_results

In [10]:
from multiprocessing import Pool


def parallel_sol(files):
    with Pool(len(files)) as pool:
        results = pool.map(get_count_sum, files)
        return sum_results(results)

In [11]:
parallel_sol(filenames)

{'1-day-or-more': 4.459439233685891,
 '15-minutes-or-less': 4.9865173969072165,
 '3-steps-or-less': 4.728792059112676,
 '30-minutes-or-less': 7.610044050452816,
 '4-hours-or-less': 10.065202927478376,
 '5-ingredients-or-less': 5.345920719916704,
 '60-minutes-or-less': 9.413654300607185,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.5116482077581974,
 'a1-sauce': 3.5280419809773695,
 'african': 4.371965990595164,
 'american': 7.575099698868723,
 'amish-mennonite': 3.560696788241698,
 'angolan': 3.4931362780914426,
 'appetizers': 6.231026464978969,
 'apples': 4.85682676038006,
 'april-fools-day': 3.5103864194773284,
 'argentine': 3.5665815193247448,
 'artichoke': 3.4960065645514224,
 'asian': 6.445874984183222,
 'asparagus': 4.047421217955213,
 'australian': 4.218603314493725,
 'austrian': 3.582217343578485,
 'avocado': 3.531110989071338,
 'bacon': 4.10665269251884,
 'baja': 3.53721200066302,
 'baked-beans': 3.4936835106382977,
 'baking': 3.6306821245618766,

In [12]:
%timeit parallel_sol(filenames)

2.59 s ± 103 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [13]:
%%file 'zad_5.py'
import pandas as pd


def target(input_q, output_q):
    while not input_q.empty():
        file = input_q.get()
        res = get_count_sum(file)
        output_q.put(res)


def get_count_sum(file):
    data = pd.read_csv(file, sep=';')
    group = data.groupby('tag')
    count = group['n_steps'].count()
    summa = group['n_steps'].sum()
    return count, summa


def sum_results(results):
    count = 0
    summa = 0
    for data in results:
        count += data[0]
        summa += data[1]
    mean = summa / count
    return mean.to_dict()

Overwriting zad_5.py


In [14]:
from zad_5 import target, get_count_sum, sum_results

In [16]:
from multiprocessing import Queue, Process


if __name__ == '__main__':
    input_q = Queue()
    output_q = Queue()
    for filename in filenames:
        input_q.put(filename)

    for i in range(int(6 / 2)):
        Process(target = target, args = (input_q, output_q)).start()

    results = []
    for i in range(len(filenames)):
        print(i)
        results.append(output_q.get())

    print(sum_results(results))

0
1
2
3
4
5
6
7
{'1-day-or-more': 4.459439233685891, '15-minutes-or-less': 4.9865173969072165, '3-steps-or-less': 4.728792059112676, '30-minutes-or-less': 7.610044050452816, '4-hours-or-less': 10.065202927478376, '5-ingredients-or-less': 5.345920719916704, '60-minutes-or-less': 9.413654300607185, 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.5116482077581974, 'a1-sauce': 3.5280419809773695, 'african': 4.371965990595164, 'american': 7.575099698868723, 'amish-mennonite': 3.560696788241698, 'angolan': 3.4931362780914426, 'appetizers': 6.231026464978969, 'apples': 4.85682676038006, 'april-fools-day': 3.5103864194773284, 'argentine': 3.5665815193247448, 'artichoke': 3.4960065645514224, 'asian': 6.445874984183222, 'asparagus': 4.047421217955213, 'australian': 4.218603314493725, 'austrian': 3.582217343578485, 'avocado': 3.531110989071338, 'bacon': 4.10665269251884, 'baja': 3.53721200066302, 'baked-beans': 3.4936835106382977, 'baking': 3.6306821245618766, 'bananas'